# 高速フーリエ変換

高速フーリエ変換は離散フーリエ変換の高速なアルゴリズム

## 1の原始$N$乗根による表現


:::{admonition} 1の原始$N$乗根

$\omega_N$を次のように定義する。

$$
\omega_N := e^{i 2 \pi / N}
$$

これは$N$次方程式$z^N-1=0$の解であり、次の関係を満たす。

$$
\omega_N^N=1, \quad \omega_N^k \neq 1, \quad k=1,2, \ldots, N-1
$$

上の式を満たす$\omega_N$を **1の原始$N$乗根** と呼ぶ。

:::

$\omega_N$を用いると、離散フーリエ変換の式

$$
f_l=\sum_{k=0}^{N-1} F_k e^{i 2 \pi k l / N}, \quad F_k=\frac{1}{N} \sum_{l=0}^{N-1} f_l e^{-i 2 \pi k l / N}
$$

は次のように書くことができる

$$
f_l=\sum_{k=0}^{N-1} F_k \omega_N^{k l}, \quad F_k=\frac{1}{N} \sum_{l=0}^{N-1} f_l \omega_N^{-k l}
$$